# Module import

In [172]:
import pandas as pd
import numpy as np

# Read the data

For now, the data are read directly in the directory where a copy of the spreadsheet has been put (in github directory). In the future, this will need to change

In [173]:
df = pd.read_excel('BDD consolidée - toutes les tables_v03.xlsx')#, dtype={'Groupe': str})

xls = pd.ExcelFile('BDD consolidée - toutes les tables_v03.xlsx')
df_companies_data = pd.read_excel(xls, 'Companies data')
df_companies_brands = pd.read_excel(xls, 'Companies brands')
df_companies_competitors = pd.read_excel(xls, 'Companies competitors')
df_global_score = pd.read_excel(xls, 'Global score display')
df_emissions_scope = pd.read_excel(xls, 'Emissions scopes description')
df_direct_complete_score = pd.read_excel(xls, 'Direct and complete score displ')
df_direct_commitment = pd.read_excel(xls, 'Direct and complete commitment ')
df_coeff_director = pd.read_excel(xls, 'Coeff directeur pour graph')
df_pos_cursors = pd.read_excel(xls, 'Positionnement des curseurs')
df_marques_by_cat = pd.read_excel(xls, 'Table Marques par catégorie')


## Brands: names, id and logo

In [174]:
df_final = df_companies_data.copy(deep=True)
list_groups = df["Group"]
df_final.columns

Index(['Group', 'Sector', 'Logo', 'Country code', 'Turnover',
       'Climate website', 'Comment', 'Global score', 'C1 direct score',
       'C1 reduction', 'C1 unit', 'C1 initial date', 'C1 final date',
       'C2 complete score', 'C2 reduction', 'C2 unit', 'C2 initial date',
       'C2 final date', 'Footnote', 'E1 score commitment direct', 'E1 phrase ',
       'E1 perimeter', 'E1 reduction', 'E1 unit', 'E1 final date',
       'E1 final date.1', 'E2 score commitment', 'E2 phrase ', 'E2 perimeter',
       'E2 reduction', 'E2 unit', 'E2 final date', 'E2 final date.1',
       'Share direct vs complet', 'Tota emissions', 'Cat 1 amount',
       'Cat 1 name', 'Cat 2 amount', 'Cat 2 name', 'Cat 3 amount',
       'Cat 3 name', 'Cat 4 amount', 'Cat 4 name', 'Cat 5 amount',
       'Cat 5 name', 'Cat 6 amount', 'Cat 6 name'],
      dtype='object')

## Create a random id using encoding

In [175]:
import base64

def encode(text):
  btext = text.encode('utf-8')[:6]
  return base64.b64encode(btext).decode("utf-8") 


df_final['company_id'] = [encode(x) for x in df_final['Group']]

In [176]:
top_5_brands_names=[]
top_5_brands_logos=[]

# Do a loop over the list of groups and filter the table df_top_brands on the group name
# Keep only the first 5 groups
# If less than 5 groups, print 'NaN' instead

for (k,group) in enumerate(list_groups):
    df_top_brands = df_companies_brands[df_companies_brands["Groupe"]==group].sort_values("Ordre")[:5]

    group_5_brand_name=[]
    group_5_brand_logo=[]
    
    for i in range(5):
        if i<len(df_top_brands):
            group_5_brand_name.append(df_top_brands["Marques"].iloc[i])
            group_5_brand_logo.append(df_top_brands["Logo"].iloc[i])
           
        else:
            group_5_brand_name.append('NaN')
            group_5_brand_logo.append('NaN')

    top_5_brands_names.append(group_5_brand_name)    
    top_5_brands_logos.append(group_5_brand_logo)    


df_final[["Top brand 1","Top brand 2","Top brand 3","Top brand 4","Top brand 5"]] = top_5_brands_names
df_final[["Logo brand 1","Logo brand 2","Logo brand 3","Logo brand 4","Logo brand 5"]] = top_5_brands_logos

## Competitors : names

In [177]:
# Just need to rename the column to match the column of final dataframe and do a merge
df_companies_competitors.rename(columns={"Groupe":"Group"}, inplace=True)
df_final = pd.merge(df_final, df_companies_competitors, how="left", on="Group")

## Comments

In [178]:
# TODO, this should come from NLP analysis

## Scores

### Pre-formatting 

#### Coeff_director

In [179]:
# format the table so that a multi-index gets back the coefficient directly
df_coeff_director = df_coeff_director.drop(columns=["degre final"])
df_coeff_director = df_coeff_director.set_index(["Value type","Courbes"])
df_coeff_director

Coeff directeur
Value type          Courbes                 
en valeur absolue   2°C              -0.0125
                    1,8°C            -0.0250
                    1,5°C            -0.0420
par unité de volume 2°C              -0.0200
                    1,8°C            -0.0400
                    1,5°C            -0.0672
par unité de valeur 2°C              -0.0700
                    1,8°C            -0.1000
                    1,5°C            -0.1500

#### Cursor position

In [180]:
df_pos_cursors = df_pos_cursors.drop(columns=["Catégorie du secteur","Intervalle","Commentaire","level"]).rename(columns={"Value type - Grandeur utilisée (évolution annuelle CO2)":"unit",
                "Borne max (évolution annuelle CO2)":"max",
                "Borne min (évolution annuelle CO2)":"min"})

df_pos_cursors = df_pos_cursors.set_index(["unit","Score"])

df_pos_cursors

max     min
unit                Score                
en valeur absolue   6.0   -0.0420 -0.0840
                    5.0   -0.0250 -0.0420
                    4.0   -0.0125 -0.0250
                    3.0    0.0000 -0.0125
                    2.0    0.0420  0.0000
                    99.0      NaN     NaN
                    1.0       NaN     NaN
par unité de volume 6.0   -0.0672 -0.1344
                    5.0   -0.0400 -0.0672
                    4.0   -0.0200 -0.0400
                    3.0    0.0000 -0.0200
                    2.0    0.0672  0.0000
                    99.0      NaN     NaN
                    1.0       NaN     NaN
par unité de valeur 6.0   -0.1500 -0.3000
                    5.0   -0.1000 -0.1500
                    4.0   -0.0700 -0.1000
                    3.0    0.0000 -0.0700
                    2.0    0.1500  0.0000
                    99.0      NaN     NaN
                    1.0       NaN     NaN

#### Display of scores

In [181]:
df_global_score = df_global_score.set_index("Global score")
df_global_score

,Color Hex,Short label,Long label,Badge
Global score,,,,
1.0,820000.0,Undisclosed,Carbon footprint not disclosed,NaN
2.0,C00000,Highly insufficient,Increasing its carbon footprint,NaN
3.0,FF8939,Insufficient,Stable ou slowly decreasing carbon footprint,NaN
4.0,FEC800,Partial,Carbon footprint partially decreasing,NaN
5.0,8CDF41,Strong,Carbon footprint strongly decreasing,NaN
6.0,0DB800,Very strong,Carbon footprint very strongly decreasing,NaN


In [182]:
df_direct_complete_score = df_direct_complete_score.set_index("Direct or complete score")
df_direct_complete_score

,Color Hex,Short label,Long label
Direct or complete score,,,
1.0,820000.0,Not reported,Not disclosed
2.0,C00000,Towards +4°C,Increasing emissions. Heading towards +4°C.
3.0,FF8939,Between +2°C and +3°C,Stable ou decreasing too slowly. Not compatibl...
4.0,FEC800,2°C,Slightly decreasing. Compatible 2°C.
5.0,8CDF41,Well-below 2°C,Strongly decreasing. Compatible well-below 2°C
6.0,0DB800,1.5°C,Very strongly decreasing. Compatible 1.5°C.
99.0,C00000,n.a. (too recent),Measure only recent


### Global score

In [183]:
# This assumes that there is always a value for global score
df_final[["global_score_hexa_color_code","global_score_short_label"]] = df_final["Global score"].apply(lambda x: df_global_score.loc[x])[["Color Hex", "Short label"]]

# TODO : logo

### Direct score

In [184]:
# This computes the various percentages depending on the scenarios along with the cursor level
C1_perc_scenarios = []
C1_direct_level = []
C1_reduc_per_year=[]
C1_final_value = []
for (k,group) in enumerate(list_groups):
    ini_date = df_final.loc[k,"C1 initial date"]
    fin_date = df_final.loc[k,"C1 final date"]
    
    if (ini_date != "n.a." and fin_date != "n.a."):
        diff_year = int(fin_date) - int(ini_date)
        
        if (df_final.loc[k,"C1 reduction"] != "n.a."):
            C1_final_value.append(100.0 + 100.0*df_final.loc[k,"C1 reduction"])
        
        
        # percentage per scenarios
        dim_perc_2 = float(100.0*diff_year*df_coeff_director.loc[df_final.loc[k,"C1 unit"], '2°C'])
        dim_perc_18 = float(100.0*diff_year*df_coeff_director.loc[df_final.loc[k,"C1 unit"], '1,8°C'])
        dim_perc_15 = float(100.0*diff_year*df_coeff_director.loc[df_final.loc[k,"C1 unit"], '1,5°C'])
        C1_perc_scenarios.append([100.0+dim_perc_2,100.0+dim_perc_18,100.0+dim_perc_15])
        
        # compute direct level of the cursor
        reduc_per_year = df_final.loc[k,"C1 reduction"] / diff_year
        min_inter = float(df_pos_cursors.loc[df_final.loc[k,"C1 unit"], df_final.loc[k,"C1 direct score"]]["min"])
        max_inter = float(df_pos_cursors.loc[df_final.loc[k,"C1 unit"], df_final.loc[k,"C1 direct score"]]["max"])
        C1_reduc_per_year.append(100.0*reduc_per_year)

        # Just to check some 'weird cases'
        if (reduc_per_year <min_inter or reduc_per_year>max_inter):
            print("problem with ",df_final.loc[k,"Group"], "; C1 score in DBB : ",df_final.loc[k,"C1 direct score"],
                 "but the annual reduction is: ", 100.0*reduc_per_year)
            
        
        if df_final.loc[k,"C1 direct score"] == 6.0 and reduc_per_year<min_inter:
            C1_direct_level.append(6.99) ## max level cursor (outside of interval)
        elif df_final.loc[k,"C1 direct score"] == 2.0 and reduc_per_year>max_inter:
            C1_direct_level.append(2.0) ## max level cursor (outside of interval)    
        else:
            # linear fit in the score interval
            C1_direct_level.append(df_final.loc[k,"C1 direct score"] + (reduc_per_year-max_inter)/(min_inter - max_inter))
    else:
        C1_perc_scenarios.append(["n.a.","n.a.","n.a."])
        C1_direct_level.append("n.a.")
        C1_reduc_per_year.append("n.a.")
        C1_final_value.append("n.a.")


df_final[["C1 2deg final","C1 1,8deg final","C1 1,5deg final"]] = C1_perc_scenarios
df_final["C1 reduc per year"] = C1_reduc_per_year
df_final["C1 direct level"] = C1_direct_level
df_final["C1 final value"] = C1_final_value


# This assumes that there is always a value for direct score and computes the display related variables
df_final[["direct_score_hexa_color_code","direct_score_short_label"]] = df_final["C1 direct score"].apply(lambda x: df_direct_complete_score.loc[x])[["Color Hex", "Short label"]]


problem with  Ecotone ; C1 score in DBB :  6.0 but the annual reduction is:  -4.066543676425535
problem with  Ferrero ; C1 score in DBB :  2.0 but the annual reduction is:  6.920748524024988
problem with  General Mills ; C1 score in DBB :  6.0 but the annual reduction is:  -4.195823573659525
problem with  Unilever ; C1 score in DBB :  6.0 but the annual reduction is:  -10.743296601975
problem with  Coty ; C1 score in DBB :  6.0 but the annual reduction is:  -11.367949999999999
problem with  L'Oréal ; C1 score in DBB :  6.0 but the annual reduction is:  -9.913487699999997
problem with  Beiersdorf ; C1 score in DBB :  6.0 but the annual reduction is:  -14.548000799999993


### C2 columns

In [185]:
# This computes the various percentages depending on the scenarios along with the cursor level
C2_perc_scenarios = []
C2_direct_level = []
C2_reduc_per_year=[]
C2_final_value = []
for (k,group) in enumerate(list_groups):
    ini_date = df_final.loc[k,"C2 initial date"]
    fin_date = df_final.loc[k,"C2 final date"]
    
    if (ini_date != "n.a." and fin_date != "n.a."):
        diff_year = int(fin_date) - int(ini_date)
        
        if (df_final.loc[k,"C2 reduction"] != "n.a."):
            C2_final_value.append(100.0 + 100.0*df_final.loc[k,"C2 reduction"])
        
        
        # percentage per scenarios
        dim_perc_2 = float(100.0*diff_year*df_coeff_director.loc[df_final.loc[k,"C2 unit"], '2°C'])
        dim_perc_18 = float(100.0*diff_year*df_coeff_director.loc[df_final.loc[k,"C2 unit"], '1,8°C'])
        dim_perc_15 = float(100.0*diff_year*df_coeff_director.loc[df_final.loc[k,"C2 unit"], '1,5°C'])
        C2_perc_scenarios.append([100.0+dim_perc_2,100.0+dim_perc_18,100.0+dim_perc_15])
        
        # compute direct level of the cursor
        reduc_per_year = df_final.loc[k,"C2 reduction"] / diff_year
        min_inter = float(df_pos_cursors.loc[df_final.loc[k,"C2 unit"], df_final.loc[k,"C2 complete score"]]["min"])
        max_inter = float(df_pos_cursors.loc[df_final.loc[k,"C2 unit"], df_final.loc[k,"C2 complete score"]]["max"])
        C2_reduc_per_year.append(100.0*reduc_per_year)

        # Just to check some 'weird cases'
        if (reduc_per_year <min_inter or reduc_per_year>max_inter):
            print("problem with ",df_final.loc[k,"Group"], "; C2 score in DBB : ",df_final.loc[k,"C2 complete score"],
                 "but the annual reduction is: ", 100.0*reduc_per_year)
            
        
        if df_final.loc[k,"C2 complete score"] == 6.0 and reduc_per_year<min_inter:
            C2_direct_level.append(6.99) ## max level cursor (outside of interval)
        elif df_final.loc[k,"C2 complete score"] == 2.0 and reduc_per_year>max_inter:
            C2_direct_level.append(2.0) ## max level cursor (outside of interval)    
        else:
            # linear fit in the score interval
            C2_direct_level.append(df_final.loc[k,"C2 complete score"] + (reduc_per_year-max_inter)/(min_inter - max_inter))
    else:
        C2_perc_scenarios.append(["n.a.","n.a.","n.a."])
        C2_direct_level.append("n.a.")
        C2_reduc_per_year.append("n.a.")
        C2_final_value.append("n.a.")


df_final[["C2 2deg final","C2 1,8deg final","C2 1,5deg final"]] = C2_perc_scenarios
df_final["C2 reduc per year"] = C2_reduc_per_year
df_final["C2 complete level"] = C2_direct_level
df_final["C2 final value"] = C2_final_value

# This assumes that there is always a value for complete score and computes the display related variables
df_final[["complete_score_hexa_color_code","complete_score_short_label"]] = df_final["C2 complete score"].apply(lambda x: df_direct_complete_score.loc[x])[["Color Hex", "Short label"]]


problem with  Bel ; C2 score in DBB :  5.0 but the annual reduction is:  -7.300000000000001
problem with  Coty ; C2 score in DBB :  3.0 but the annual reduction is:  0.10000000000000009


In [186]:
df_final.columns

Index(['Group', 'Sector', 'Logo', 'Country code', 'Turnover',
       'Climate website', 'Comment', 'Global score', 'C1 direct score',
       'C1 reduction', 'C1 unit', 'C1 initial date', 'C1 final date',
       'C2 complete score', 'C2 reduction', 'C2 unit', 'C2 initial date',
       'C2 final date', 'Footnote', 'E1 score commitment direct', 'E1 phrase ',
       'E1 perimeter', 'E1 reduction', 'E1 unit', 'E1 final date',
       'E1 final date.1', 'E2 score commitment', 'E2 phrase ', 'E2 perimeter',
       'E2 reduction', 'E2 unit', 'E2 final date', 'E2 final date.1',
       'Share direct vs complet', 'Tota emissions', 'Cat 1 amount',
       'Cat 1 name', 'Cat 2 amount', 'Cat 2 name', 'Cat 3 amount',
       'Cat 3 name', 'Cat 4 amount', 'Cat 4 name', 'Cat 5 amount',
       'Cat 5 name', 'Cat 6 amount', 'Cat 6 name', 'company_id', 'Top brand 1',
       'Top brand 2', 'Top brand 3', 'Top brand 4', 'Top brand 5',
       'Logo brand 1', 'Logo brand 2', 'Logo brand 3', 'Logo brand 4',
      

# Renaming and final modifications

In [187]:
df_data_viz = df_final.copy(deep=True)
df_data_viz.columns

Index(['Group', 'Sector', 'Logo', 'Country code', 'Turnover',
       'Climate website', 'Comment', 'Global score', 'C1 direct score',
       'C1 reduction', 'C1 unit', 'C1 initial date', 'C1 final date',
       'C2 complete score', 'C2 reduction', 'C2 unit', 'C2 initial date',
       'C2 final date', 'Footnote', 'E1 score commitment direct', 'E1 phrase ',
       'E1 perimeter', 'E1 reduction', 'E1 unit', 'E1 final date',
       'E1 final date.1', 'E2 score commitment', 'E2 phrase ', 'E2 perimeter',
       'E2 reduction', 'E2 unit', 'E2 final date', 'E2 final date.1',
       'Share direct vs complet', 'Tota emissions', 'Cat 1 amount',
       'Cat 1 name', 'Cat 2 amount', 'Cat 2 name', 'Cat 3 amount',
       'Cat 3 name', 'Cat 4 amount', 'Cat 4 name', 'Cat 5 amount',
       'Cat 5 name', 'Cat 6 amount', 'Cat 6 name', 'company_id', 'Top brand 1',
       'Top brand 2', 'Top brand 3', 'Top brand 4', 'Top brand 5',
       'Logo brand 1', 'Logo brand 2', 'Logo brand 3', 'Logo brand 4',
      

In [188]:
df_data_viz = df_data_viz.rename(columns={
    "Group": "company_name",
    "C1 final value": "c1_final_value",
    "C1 2deg final": "c1_2deg_final",
    "C1 1,8deg final": "c1_1_8deg_final",
    "C1 1,5deg final": "c1_1_5deg_final",
    "C1 initial date": "C1_initial_date",
    "C1 final date": "C1_final_date",
    "C2 final value": "c2_final_value",
    "C2 2deg final": "c2_2deg_final",
    "C2 1,8deg final": "c2_1_8deg_final",
    "C2 1,5deg final": "c2_1_5deg_final",
    "C2 initial date": "C2_initial_date",
    "C2 final date": "C2_final_date",
    "Comment": "comment",
    "Global score": "global_score",
    "C1 direct score": "direct_score",
    "C2 complete score": "complete_score",
    
})

In [189]:
df_data_viz[["company_name","company_id","c1_final_value","c1_2deg_final","c1_1_8deg_final","c1_1_5deg_final","C1_final_date","C1_initial_date"]]

,company_name,company_id,c1_final_value,c1_2deg_final,c1_1_8deg_final,c1_1_5deg_final,C1_final_date,C1_initial_date
0,Andros,QW5kcm9z,n.a.,n.a.,n.a.,n.a.,n.a.,n.a.
1,Associated British Foods (alim.),QXNzb2Np,87.439144,95.0,90.0,83.2,2020,2016
2,Bel,QmVs,96.059601,95.0,90.0,83.2,2020,2016
3,Danone,RGFub25l,62.061092,93.75,87.5,79.0,2020,2015
4,Ecotone,RWNvdG9u,67.467651,90.0,80.0,66.4,2018,2010
5,Ferrero,RmVycmVy,127.682994,95.0,90.0,83.2,2019,2015
6,General Mills,R2VuZXJh,79.020882,93.75,87.5,79.0,2020,2015
7,Hero,SGVybw==,67.965667,82.0,64.0,39.52,2019,2010
8,Kellogg,S2VsbG9n,71.328328,95.0,90.0,83.2,2019,2015
9,Mars,TWFycw==,78.074896,95.0,90.0,83.2,2019,2015


In [190]:
df_data_viz[["company_name","company_id","c2_final_value","c2_2deg_final","c2_1_8deg_final","c2_1_5deg_final","C2_final_date","C2_initial_date"]]


,company_name,company_id,c2_final_value,c2_2deg_final,c2_1_8deg_final,c2_1_5deg_final,C2_final_date,C2_initial_date
0,Andros,QW5kcm9z,n.a.,n.a.,n.a.,n.a.,n.a.,n.a.
1,Associated British Foods (alim.),QXNzb2Np,n.a.,n.a.,n.a.,n.a.,n.a.,n.a.
2,Bel,QmVs,85.4,96.0,92.0,86.56,2019,2017
3,Danone,RGFub25l,72.5,90.0,80.0,66.4,2020,2015
4,Ecotone,RWNvdG9u,n.a.,n.a.,n.a.,n.a.,n.a.,n.a.
5,Ferrero,RmVycmVy,107.5,96.25,92.5,87.4,2019,2016
6,General Mills,R2VuZXJh,83.0,87.5,75.0,58.0,2020,2010
7,Hero,SGVybw==,n.a.,n.a.,n.a.,n.a.,n.a.,n.a.
8,Kellogg,S2VsbG9n,95.2,95.0,90.0,83.2,2019,2015
9,Mars,TWFycw==,98.0,95.0,90.0,83.2,2019,2015


In [191]:
df_data_viz[["global_score_hexa_color_code","global_score_short_label","comment","global_score"]]

,global_score_hexa_color_code,global_score_short_label,comment,global_score
0,820000.0,Undisclosed,Un culte du secret incompatible avec les enjeu...,1.0
1,FF8939,Insufficient,Seuls ~10% de son empreinte carbone complète s...,3.0
2,FEC800,Partial,"Réduction encore partielle, malgré une forte r...",4.0
3,0DB800,Very strong,Très fortes réductions de toutes ses émissions...,6.0
4,8CDF41,Strong,Très forte réduction de ses émissions propres....,5.0
5,C00000,Highly insufficient,"Des émissions toujours en hausse. Depuis 2010,...",2.0
6,8CDF41,Strong,Haut niveau d'action initié. A même renforcé s...,5.0
7,FEC800,Partial,Réduction significative de ses émissions propr...,4.0
8,FEC800,Partial,"Accélération des efforts à partir de 2015, ave...",4.0
9,FEC800,Partial,"Accélération des efforts à partir de 2015, ave...",4.0


In [192]:
df_data_viz[["direct_score","direct_score_hexa_color_code","direct_score_short_label"]]

,direct_score,direct_score_hexa_color_code,direct_score_short_label
0,1.0,820000.0,Not reported
1,5.0,8CDF41,Well-below 2°C
2,3.0,FF8939,Between +2°C and +3°C
3,6.0,0DB800,1.5°C
4,6.0,0DB800,1.5°C
5,2.0,C00000,Towards +4°C
6,6.0,0DB800,1.5°C
7,4.0,FEC800,2°C
8,6.0,0DB800,1.5°C
9,6.0,0DB800,1.5°C


In [193]:
df_data_viz[["complete_score","complete_score_hexa_color_code","complete_score_short_label"]]

,complete_score,complete_score_hexa_color_code,complete_score_short_label
0,1.0,820000.0,Not reported
1,1.0,820000.0,Not reported
2,5.0,8CDF41,Well-below 2°C
3,5.0,8CDF41,Well-below 2°C
4,1.0,820000.0,Not reported
5,2.0,C00000,Towards +4°C
6,4.0,FEC800,2°C
7,1.0,820000.0,Not reported
8,3.0,FF8939,Between +2°C and +3°C
9,3.0,FF8939,Between +2°C and +3°C
